# Validating DSS File Paths and Data Availability

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
# Set USE_LOCAL_SOURCE based on your setup:
#   True  = Use local source code (for developers editing ras-commander)  
#   False = Use pip-installed package (for users)
# =============================================================================

USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

# -----------------------------------------------------------------------------
if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)  # Parent of examples/ = repo root
    if local_path not in sys.path:
        sys.path.insert(0, local_path)  # Insert at position 0 = highest priority
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander (will use local or pip based on toggle above)
from ras_commander import *
from ras_commander.dss import RasDss
from ras_commander.validation_base import ValidationSeverity

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

---

## Overview

This notebook demonstrates the DSS pathname validation framework in ras-commander. DSS (Data Storage System) files are commonly used to store boundary conditions for HEC-RAS models. Validating pathnames and data availability is critical for ensuring model runs will succeed.

The validation framework provides:
- Format validation (pathname structure)
- File existence checks
- Pathname catalog verification
- Time series data availability checks
- Comprehensive validation reports

We'll use the **BaldEagleCrkMulti2D** example project which includes DSS boundary conditions.

---

## Extract Example Project

In [ ]:
# Extract BaldEagleCrkMulti2D project (contains DSS boundary files)
project_path = RasExamples.extract_project("BaldEagleCrkMulti2D")
print(f"\nProject extracted to: {project_path}")

In [ ]:
# Find DSS files in the project
from pathlib import Path

dss_files = list(project_path.glob("*.dss"))
print(f"Found {len(dss_files)} DSS file(s):")
for dss_file in dss_files:
    print(f"  - {dss_file.name} ({dss_file.stat().st_size / 1024:.2f} KB)")

In [ ]:
# Use first DSS file for validation examples
if dss_files:
    test_dss_file = dss_files[0]
    print(f"Using DSS file: {test_dss_file.name}")
    
    # Get catalog to see available pathnames
    catalog = RasDss.get_catalog(test_dss_file)
    print(f"\nDSS file contains {len(catalog)} pathname(s):")
    for i, pathname in enumerate(catalog[:10], 1):  # Show first 10
        print(f"  {i}. {pathname}")
    if len(catalog) > 10:
        print(f"  ... and {len(catalog) - 10} more")
else:
    print("⚠️ No DSS files found in project - will use synthetic examples")
    test_dss_file = project_path / "boundary.dss"  # For examples

---

## 1. Format Validation: `check_pathname_format()`

DSS pathnames must follow the format: `//A/B/C/D/E/F/`

Where:
- **A** = Basin/Project
- **B** = Location
- **C** = Parameter (e.g., FLOW, STAGE)
- **D** = Date
- **E** = Interval (e.g., 1HOUR, 15MIN)
- **F** = Scenario/Version

In [ ]:
# Example 1: Valid pathname format
valid_pathname = "//BASIN/LOCATION/FLOW/01JAN2020/1HOUR/OBS/"

result = RasDss.check_pathname_format(valid_pathname)
print(f"Pathname: {valid_pathname}")
print(f"Result: {result}")
print(f"\nDetails:")
for key, value in result.details.items():
    print(f"  {key}: {value}")

In [ ]:
# Example 2: Invalid pathname - missing leading //
invalid_pathname1 = "/BASIN/LOCATION/FLOW/01JAN2020/1HOUR/OBS/"

result = RasDss.check_pathname_format(invalid_pathname1)
print(f"Pathname: {invalid_pathname1}")
print(f"Result: {result}")
print(f"Passed: {result.passed}")
print(f"Severity: {result.severity.value}")

In [ ]:
# Example 3: Invalid pathname - wrong number of parts
invalid_pathname2 = "//BASIN/LOCATION/FLOW/01JAN2020/"  # Missing parts

result = RasDss.check_pathname_format(invalid_pathname2)
print(f"Pathname: {invalid_pathname2}")
print(f"Result: {result}")
print(f"\nDetails:")
for key, value in result.details.items():
    print(f"  {key}: {value}")

In [ ]:
# Example 4: Valid but with empty parts (warning)
pathname_with_empty = "//BASIN/LOCATION/FLOW//1HOUR/OBS/"  # Empty date part

result = RasDss.check_pathname_format(pathname_with_empty)
print(f"Pathname: {pathname_with_empty}")
print(f"Result: {result}")
print(f"Passed: {result.passed} (with warnings)")
print(f"Severity: {result.severity.value}")

---

## 2. File Existence: `check_file_exists()`

Validates that the DSS file exists and is accessible.

In [ ]:
# Example 1: Valid file (exists)
if dss_files:
    result = RasDss.check_file_exists(test_dss_file)
    print(f"File: {test_dss_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")

In [ ]:
# Example 2: Invalid file (doesn't exist)
nonexistent_file = project_path / "nonexistent.dss"

result = RasDss.check_file_exists(nonexistent_file)
print(f"File: {nonexistent_file.name}")
print(f"Result: {result}")
print(f"Passed: {result.passed}")
print(f"Severity: {result.severity.value} (CRITICAL - blocks all further checks)")

---

## 3. Pathname Existence: `check_pathname_exists()`

Validates that a specific pathname exists in the DSS file catalog.

In [ ]:
# Example 1: Pathname exists (use actual pathname from catalog)
if dss_files and len(catalog) > 0:
    existing_pathname = catalog[0]  # First pathname in catalog
    
    result = RasDss.check_pathname_exists(test_dss_file, existing_pathname)
    print(f"Pathname: {existing_pathname}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")

In [ ]:
# Example 2: Pathname doesn't exist
if dss_files:
    missing_pathname = "//BASIN/NOWHERE/FLOW/01JAN2020/1HOUR/MISSING/"
    
    result = RasDss.check_pathname_exists(test_dss_file, missing_pathname)
    print(f"Pathname: {missing_pathname}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        if key == 'similar_paths':
            print(f"  {key}:")
            for path in value:
                print(f"    - {path}")
        else:
            print(f"  {key}: {value}")

---

## 4. Data Availability: `check_data_availability()`

Validates that time series data exists and optionally checks date range coverage.

In [ ]:
# Example 1: Check data availability (without date range)
if dss_files and len(catalog) > 0:
    data_pathname = catalog[0]
    
    result = RasDss.check_data_availability(test_dss_file, data_pathname)
    print(f"Pathname: {data_pathname}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")

In [ ]:
# Example 2: Check data availability with expected date range
if dss_files and len(catalog) > 0:
    from datetime import datetime
    
    data_pathname = catalog[0]
    
    # Get actual date range from data
    df = RasDss.read_timeseries(test_dss_file, data_pathname)
    actual_start = df.index.min()
    actual_end = df.index.max()
    
    print(f"Actual data range: {actual_start} to {actual_end}")
    
    # Check with expected range that matches actual (should pass)
    result = RasDss.check_data_availability(
        test_dss_file,
        data_pathname,
        expected_start=actual_start,
        expected_end=actual_end
    )
    print(f"\nValidation with matching range:")
    print(f"Result: {result}")

In [ ]:
# Example 3: Check with expected range beyond actual data (warning)
if dss_files and len(catalog) > 0:
    from datetime import datetime, timedelta
    
    data_pathname = catalog[0]
    df = RasDss.read_timeseries(test_dss_file, data_pathname)
    actual_end = df.index.max()
    
    # Request data extending beyond what's available
    extended_end = actual_end + timedelta(days=365)
    
    result = RasDss.check_data_availability(
        test_dss_file,
        data_pathname,
        expected_end=extended_end
    )
    print(f"Validation with extended range:")
    print(f"Result: {result}")
    print(f"Severity: {result.severity.value} (WARNING - data incomplete)")

---

## 5. Comprehensive Validation: `check_pathname()`

Performs all validation checks in sequence and returns a comprehensive report.

In [ ]:
# Example 1: Comprehensive validation - valid pathname
if dss_files and len(catalog) > 0:
    valid_pathname = catalog[0]
    
    report = RasDss.check_pathname(
        dss_file=test_dss_file,
        pathname=valid_pathname
    )
    
    print(f"Comprehensive validation for: {valid_pathname}")
    print(f"\nReport: {report}")
    print(f"Is valid: {report.is_valid}")
    print(f"Has warnings: {report.has_warnings}")
    
    # Print formatted report
    report.print_report(show_passed=True)

In [ ]:
# Example 2: Comprehensive validation - invalid pathname
if dss_files:
    invalid_pathname = "//BASIN/MISSING/FLOW/01JAN2020/1HOUR/NONE/"
    
    report = RasDss.check_pathname(
        dss_file=test_dss_file,
        pathname=invalid_pathname
    )
    
    print(f"Comprehensive validation for: {invalid_pathname}")
    print(f"\nIs valid: {report.is_valid}")
    
    # Print only failures (default behavior)
    report.print_report(show_passed=False)

In [ ]:
# Example 3: Filter results by severity
if dss_files and len(catalog) > 0:
    pathname = catalog[0]
    
    report = RasDss.check_pathname(test_dss_file, pathname)
    
    # Get only errors
    errors = report.get_results_by_severity(ValidationSeverity.ERROR)
    print(f"Errors found: {len(errors)}")
    for error in errors:
        print(f"  - {error}")
    
    # Get only warnings
    warnings = report.get_results_by_severity(ValidationSeverity.WARNING)
    print(f"\nWarnings found: {len(warnings)}")
    for warning in warnings:
        print(f"  - {warning}")
    
    # Get all failed checks
    failed = report.get_failed_checks()
    print(f"\nFailed checks: {len(failed)}")

---

## 6. Boolean Convenience Methods

For quick checks without detailed reports.

In [ ]:
# Example 1: Quick pathname format check
pathnames_to_check = [
    "//BASIN/LOC/FLOW/01JAN2020/1HOUR/OBS/",  # Valid
    "/BASIN/LOC/FLOW/01JAN2020/1HOUR/OBS/",   # Invalid - missing //
    "//BASIN/LOC/FLOW/01JAN2020/",           # Invalid - too few parts
]

for pathname in pathnames_to_check:
    is_valid = RasDss.is_valid_pathname(pathname)
    status = "✓ VALID" if is_valid else "✗ INVALID"
    print(f"{status}: {pathname}")

In [ ]:
# Example 2: Quick pathname availability check
if dss_files and len(catalog) > 0:
    # Check if pathname is available (file exists, pathname exists, has data)
    existing_pathname = catalog[0]
    missing_pathname = "//BASIN/NOWHERE/FLOW/01JAN2020/1HOUR/NONE/"
    
    print(f"Checking pathname availability:\n")
    
    is_available1 = RasDss.is_pathname_available(test_dss_file, existing_pathname)
    print(f"{'✓' if is_available1 else '✗'} {existing_pathname}")
    print(f"  Available: {is_available1}\n")
    
    is_available2 = RasDss.is_pathname_available(test_dss_file, missing_pathname)
    print(f"{'✓' if is_available2 else '✗'} {missing_pathname}")
    print(f"  Available: {is_available2}")

---

## 7. Practical Use Case: Pre-Flight Check for Model Run

Validate all boundary condition pathnames before executing a HEC-RAS plan.

In [ ]:
# Simulate pre-flight check for multiple boundary conditions
if dss_files and len(catalog) > 0:
    print("=" * 80)
    print("PRE-FLIGHT CHECK: Boundary Condition Validation")
    print("=" * 80)
    
    # Use all pathnames from catalog (or subset if many)
    pathnames_to_validate = catalog[:3] if len(catalog) > 3 else catalog
    
    all_valid = True
    
    for i, pathname in enumerate(pathnames_to_validate, 1):
        print(f"\n[{i}/{len(pathnames_to_validate)}] Validating: {pathname}")
        
        # Quick check
        is_available = RasDss.is_pathname_available(test_dss_file, pathname)
        
        if is_available:
            print(f"  ✓ PASS")
        else:
            print(f"  ✗ FAIL - running detailed diagnostics...")
            report = RasDss.check_pathname(test_dss_file, pathname)
            report.print_report(show_passed=False)
            all_valid = False
    
    print("\n" + "=" * 80)
    if all_valid:
        print("✓ PRE-FLIGHT CHECK PASSED - All boundary conditions valid")
        print("  Ready to execute HEC-RAS plan")
    else:
        print("✗ PRE-FLIGHT CHECK FAILED - Fix boundary conditions before running")
    print("=" * 80)

---

## Cleanup

In [ ]:
# Clean up extracted project
import shutil

if project_path.parent.name == "example_projects":
    shutil.rmtree(project_path.parent, ignore_errors=True)
    print("✓ Cleaned up example projects")

---

## Summary

This notebook demonstrated the DSS validation framework:

**Individual Validation Methods**:
- `check_pathname_format()` - Validates DSS pathname structure
- `check_file_exists()` - Verifies file accessibility
- `check_pathname_exists()` - Confirms pathname in catalog
- `check_data_availability()` - Validates time series coverage

**Comprehensive Validation**:
- `check_pathname()` - Runs all checks, returns ValidationReport

**Boolean Conveniences**:
- `is_valid_pathname()` - Quick format check
- `is_pathname_available()` - Quick availability check

**Validation Reports**:
- Filter by severity (INFO, WARNING, ERROR, CRITICAL)
- Get failed checks
- Print formatted reports

Use these tools to ensure DSS boundary conditions are valid before running HEC-RAS models!